In [ ]:
import numpy as np
from PIL import Image

from keras.layers import merge, Input, Conv2D, MaxPooling2D, UpSampling2D, Cropping2D, Flatten, Dense, BatchNormalization, Dropout
from keras.models import Model
from keras.layers.merge import concatenate
from keras.optimizers import Adam
import os


In [ ]:
def get_crop_shape(target, refer):
        # width, the 3rd dimension
        cw = (target.get_shape()[2] - refer.get_shape()[2]).value
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1]).value
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

In [ ]:
def get_unet(ncha, img_rows, img_cols):

    inputs = Input((img_rows, img_cols, ncha))


    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    print "conv1 shape:",conv1.shape
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    print "conv1 shape:",conv1.shape
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    print "pool1 shape:",pool1.shape

    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    print "conv2 shape:",conv2.shape
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    print "conv2 shape:",conv2.shape
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    print "pool2 shape:",pool2.shape

    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    print "conv3 shape:",conv3.shape
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    print "conv3 shape:",conv3.shape
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    print "pool3 shape:",pool3.shape

    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = [inputs], outputs = [conv10])

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [ ]:
model = get_unet(3, 512, 512)
adam = Adam()
model.compile(adam, loss=['binary_crossentropy'], metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# imgs_train = [np.expand_dims(np.array(Image.open('/root/data/hackathon/building_massa_dataset/images/train/22678915_15.tiff')), axis=0)]
# imgs_mask_train = [np.expand_dims(np.array(Image.open('/root/data/hackathon/building_massa_dataset/labels/train/22678915_15.tif'))[6:-6, 6:-6], axis=0)]

In [ ]:
batch_size = 1

In [ ]:
train_steps = 1000//batch_size
val_steps = 100//batch_size

In [ ]:
def myGenerator(file_paths, steps_per_epoch, BATCH_SIZE, INPUT_SHAPE):
    i = 0
    while True:
        x_batch = np.empty((BATCH_SIZE, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]))
        y_batch = np.empty((BATCH_SIZE, INPUT_SHAPE[0], INPUT_SHAPE[1], 1))
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            # x_batch[ind,...] = np.array(Image.open(np.random.choice(id2image[rand_class])).convert('RGB').resize((INPUT_SHAPE[0], INPUT_SHAPE[1])))
            # y_batch[ind,...] = get_1_hot(classes.index(rand_class), classes)
            # pick a random image
            f = np.random.choice(file_paths)
            x_batch[ind,...] = np.array(Image.open(f))[:512, :512, :]
            # pick the corresponding mask
            ftruth = f.replace('images', 'labels')
            ftruth = ftruth[:-1]
            mask = np.expand_dims(np.array(Image.open(ftruth))[:512, :512, 0], axis=2)
            mask[mask==255]=1
            y_batch[ind,...] = mask
        # x_batch = seq.augment_images(x_batch)
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [ ]:
train = [os.path.join('/root/data/hackathon/building_massa_dataset/images/train/', f ) for f in os.listdir('/root/data/hackathon/building_massa_dataset/images/train/') if f.endswith('.tiff')]
val = [os.path.join('/root/data/hackathon/building_massa_dataset/images/valid/', f ) for f in os.listdir('/root/data/hackathon/building_massa_dataset/images/valid/') if f.endswith('.tiff')]

In [ ]:
train_generator = myGenerator(train, train_steps, batch_size, (512, 512, 3))
validation_generator = myGenerator(val, val_steps, batch_size, (512, 512, 3))

In [ ]:
# start training
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=train_steps,
        epochs=30,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=val_steps)

In [ ]:
# model.fit_generator(imgs_train, 
#                     imgs_mask_train, 
#                     batch_size=4, 
#                     nb_epoch=10, 
#                     verbose=1,
#                     validation_split=0.2, 
#                     shuffle=True)

In [ ]:
test = model.predict(np.expand_dims(np.array(Image.open('/root/data/hackathon/building_massa_dataset/images/train/22678915_15.tiff'))[:512, :512, :], axis=0))

In [ ]:
tru = test[0,...]

In [ ]:
tru[tru>0.5] = 1

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.